### **Automação para Manter o Setor de Compras Alinhado ao Mercado Internacional**  
Este projeto tem como objetivo criar uma automação que mantenha o setor de compras sempre atualizado com as tendências do mercado internacional. A ideia é avaliar constantemente se os preços de itens como café, ouro, milho, entre outros, estão dentro de um patamar acessível para compra. Usarei o site MelhorCâmbio, conhecido pela simplicidade e consultas em tempo real, para realizar essas análises. Ao final de cada busca, será gerado um relatório por email para o setor, já destacando se a compra é viável ou não.

In [146]:
# Usaremos o selenium precisamos baixar o 'webdriver' > basta ver no site ofícial
# baixar webdriver = chromedriver

# Importacao importante
from selenium import webdriver
import pandas as pd
import unicodedata
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
 

# Bibliotecas para enviar email
import credenciais
import smtplib
from email.message import EmailMessage
from email.mime.base import MIMEBase
from email import encoders
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

### Importando a base de dados

In [152]:
# Chamando o navegador
navegador = webdriver.Chrome()
navegador.get('https://www.melhorcambio.com/Boi-hoje')
navegador.maximize_window()# maximiza tela

tabela_melhorcambio = pd.read_excel('melhorcambio.xlsx')
display(tabela_melhorcambio.head())

,Produto,Preço Ideal,Preço Atual,Cenário De Compra
0,Milho,85.32,NaN,NaN
1,Soja,163.59,NaN,NaN
2,Boi,282.20,NaN,NaN
3,Petróleo,424.37,NaN,NaN
4,Algodão,497.76,NaN,NaN


### Iniciarei uma busca no site "Melhor Câmbio" para consultar todos os seus produtos disponíveis.

In [ ]:
for linha in tabela_melhorcambio.index:

    # Para acessar o site "Melhor Câmbio" (https://www.melhorcambio.com/produtos-hoje)
    # Será utilizada uma f-string para criar um link dinâmico, permitindo acessar informações de diferentes produtos.

    # Agora o produto será normalizado para lidar com caracteres especiais, garantindo consistência e padronização nos tabela_melhorcambio.

    produto = tabela_melhorcambio.loc[linha,'Produto']
    lin = f'https://www.melhorcambio.com/{produto}-hoje'

    # - 1º Forma retirar caracter especial - Forma especifica

    # produto=produto.replace('ó','o').replace('é','e').replace('ã','a').replace('ú','u').replace('á','a').replace('ç','c')

    # - 2º Forma para qualquer caracter especial - Forma direta
    link = unicodedata.normalize("NFKD", lin).encode(encoding="ascii",errors="ignore")
    
    # Saída será essa>>> b'https://www.melhorcambio.com/produtos-hoje'
    link = str(link)

    # Limpando o link
    link = link.replace("b'",'').replace("'",'')
    
    # Busca do link formatado
    navegador.get(link)
    
    # Pegando o preço atual do produto
    preco = navegador.find_element('xpath', '//*[@id="comercial"]').get_attribute('value')
    
    # Saída será essa > '23.322,333' isso é uma 'string' 
    # Irei mudar esse texo para numero de ponto flutuante > '23322.333'
    
    preco = preco.replace('.','').replace(',','.')
    preco = float(preco)
    
    # Atualizando o preço na base de tabela_melhorcambio
    tabela_melhorcambio.loc[linha,'Preço Atual'] = preco

display(tabela_melhorcambio.head()) 

navegador.quit()

,Produto,Preço Ideal,Preço Atual,Cenário De Compra
0,Milho,85.32,73.20,NaN
1,Soja,163.59,140.90,NaN
2,Boi,282.20,323.70,NaN
3,Petróleo,424.37,433.10,NaN
4,Algodão,497.76,411.61,NaN


### Quais produtos comprar?

In [149]:
tabela_melhorcambio['Cenário De Compra'] = tabela_melhorcambio['Preço Atual'] < tabela_melhorcambio['Preço Ideal']
display(tabela_melhorcambio.head())

,Produto,Preço Ideal,Preço Atual,Cenário De Compra
0,Milho,85.32,73.20,True
1,Soja,163.59,140.90,True
2,Boi,282.20,323.70,False
3,Petróleo,424.37,433.10,False
4,Algodão,497.76,411.61,True


### Base de dados atualizada salva ( para anexo )

Caso não exista irar criar uma nova


In [150]:
# 'index=False' - retira a coluna de index
tabela_melhorcambio.to_excel('melhorcambio_atualizado.xlsx',index=False)

### Enviar email para a setor de compras

In [ ]:
def enviar_anexo_por_email(caminho):
    with open(caminho_arquivo, 'rb') as arquivo:
        # Cria uma parte MIME para o anexo.
        part = MIMEBase('application', 'octet-stream')
        part.set_payload(arquivo.read())
        encoders.encode_base64(part)  # Codifica o arquivo em base64 para envio.

        # Define o cabeçalho do anexo.
        part.add_header(
            'Content-Disposition',
            f'attachment; filename="{caminho_arquivo}"'  # Nome do arquivo no email.
        )

        # Adiciona o anexo à mensagem.
        msg.attach(part)
        
def conectar_fechar_conexao(msg,senha):
    # Conecta ao servidor SMTP.
    s = smtplib.SMTP('smtp.gmail.com', 587)
    s.starttls()  # Inicia uma conexão segura.

    # Faz login com o email e a senha.
    s.login(msg['From'], senha)

    # Envia o email com o anexo.
    s.sendmail(msg['From'], msg['To'], msg.as_string())

    # Finaliza a conexão.
    s.quit()

def cor_cenario(cenario):
    if str(cenario).lower() == 'true':
        cor_cenario = 'green'
    else:
        cor_cenario = 'red'
    return cor_cenario

linhas = """"""

for cenario in tabela_melhorcambio.values:
    linha = f"""
    <tr>
        <td>{cenario[0]}</td>
        <td style="text-align: center">R${cenario[1]}</td>
        <td style="text-align: center">R${cenario[2]}</td>
        <td style="text-align: center"><font color={cor_cenario(cenario[3])}>◙</font></td>
    </tr>
    """
    linhas = linhas + linha


# Configura a mensagem como uma mensagem multipart (HTML + anexo).
msg = MIMEMultipart()

# Define o assunto do email.
msg['Subject'] = "Relatório atualizado site Melhor Câmbio"

# Define o remetente.
msg['From'] = credenciais.remetente

# Define o destinatário. 
msg['To'] = credenciais.destinatario

# Obtém a senha do remetente. 
# Essa senha é encontrada no gmail do remetente pesquise por "Suas senhas de app" cria e cole aqui
password = credenciais.senha

# Adiciona o corpo do email como texto ou HTML.

cabecario = """
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Tabela Melhor Câmbio</title>
<style>
    /* Adiciona margem externa para a tabela */
    table {
        margin: 20px auto; /* Centraliza horizontalmente com margem superior/inferior de 20px */
        border-collapse: collapse; /* Remove espaçamento entre bordas */
        width: 80%; /* Controla a largura da tabela */
    }

    /* Adiciona bordas e espaçamento interno nas células */
    th, td {
        padding: 10px; /* Espaçamento interno */
        border: 1px solid black; /* Adiciona borda */
        text-align: center; /* Centraliza o texto */
    }

    /* Personaliza o cabeçalho */
    th {
        background-color: #f2f2f2; /* Cor de fundo do cabeçalho */
        font-weight: bold; /* Deixa o texto em negrito */
    }
</style>
</head>
"""

corpo_email = f"""<p>Bom dia, equipe do Setor de Compras,</p>

<p>Segue em anexo o relatório atualizado com os preços coletados no site Melhor Câmbio.</p>

<table>
<tr>
    <th>Produto</th>
    <th>Preço Ideal</th>
    <th>Preço Atual</th>
    <th>Cenário De Compra</th>
</tr>

{linhas}

</table>

<br>

<p>Também anexei uma planilha completa com todos os dados coletados para auxiliar na análise de viabilidade das compras.</p>

<p>Estou à disposição para esclarecer qualquer dúvida ou fornecer informações adicionais.</p>

<p>Atenciosamente,<br>Carlos Morais</p>
"""

msg.attach(MIMEText(cabecario + corpo_email, 'html'))

# Adiciona um arquivo em anexo.
caminho_arquivo = "melhorcambio_atualizado.xlsx"  # Substitua pelo caminho do arquivo desejado.

# Configurar o anexo e formatar para o envio
enviar_anexo_por_email(caminho_arquivo)

# Conectar com o servidor enviar a mensagem e depois fechar a conexao
conectar_fechar_conexao(msg,password)